In [1]:
import numpy as np
from hand_sym import rotation
import sympy as sy

%load_ext autoreload
%autoreload 2

In [2]:
q_list = ['q_0', 'q_1', 'q_2', 'q_3', 'q_4']  # finger placement, joints of robot
q = sy.symbols('q_:5')   # joint symbols

l = sy.symbols('l_:5')   # link lengths
r = sy.symbols('r_:2')   # radius for capsule and sphere

dof = len(q_list)
T = sy.eye(4)
axes = [[0,0,1], [0,0,1],[0, 1, 0],[0, 1, 0],[0, 1,0]]

T_centers=[]  # the center of each link
T_all = []   # finger tip
for i in range(dof):
    T = T * rotation(q[i], axes[i])
    T_translation = sy.eye(4)
    T_translation[:3, 3] = [l[i],0,0]    
    T  = T * T_translation   # fingertip pose
    T_translation_c = sy.eye(4)
    T_translation_c[:3, 3] = [-l[i]/2,0,0]  
    T_c = T * T_translation_c # link center
    T_centers.append(T_c)
    T_all.append(T)



In [3]:
len(T_all)

5

In [4]:
x = sy.symbols('x_:3')

In [5]:
### choose which two links for pinch grasp
i = 4  # i-th link for the first finger
j = 4  # j-th link for the second finger

for grasping_link_index in [[2,2], [3,3], [4,4], [3,4]]:
    i = grasping_link_index[0]
    j = grasping_link_index[1]
    p = sy.symbols('p_:5')   # joint symbols for second finger
    m = sy.symbols('m_:5')   # link lengths for second finger
    p2q = dict((q[tmp], p[tmp]) for tmp in range(dof))
    l2m = dict((l[tmp], m[tmp]) for tmp in range(dof))
    ql2qm = p2q | l2m
    
    T_all_j = [T.subs(ql2qm) for T in T_all]
    # x, y, z = sy.symbols('x:z')  # the sphere center
    x = sy.symbols('x_:3')
    M = sy.Matrix(x)
    
    ### get A B C D points
    A = T_all[i-1][:3, 3]
    B = T_all[i][:3, 3]
    C = T_all_j[j-1][:3, 3]
    D = T_all_j[j][:3, 3]
    
    ### get N1, N2 points, the perpendicular intersections from M to AB, and M to CD
    n1 = sy.symbols('n_1')
    Eq_n1 =  (B-A).dot(M - (A + (B-A) * n1))
    n1_val = sy.solve(Eq_n1, n1, dict=False)[0]
    N1 = A + (B-A) * n1_val
    
    n2 = sy.symbols('n_2')
    Eq_n2 =  (D-C).dot(M - (C + (D-C) * n2))
    n2_val = sy.solve(Eq_n2, n2, dict=False)[0]
    N2 = C + (D-C) * n2_val
    
    import pickle
    N1_N2_name = 'N1_N2_' + str(i)+ str(j) + '.txt'
    with open(N1_N2_name, 'wb') as f:
        pickle.dump([N1,N2], f)

In [6]:
with open(N1_N2_name, 'rb') as f:
    N1_N2 = pickle.load(f)

In [7]:
N1_N2[1]

Matrix([
[m_0*cos(p_0) + m_1*(-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1)) + m_2*(-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*cos(p_2) + m_3*(-(-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*sin(p_2)*sin(p_3) + (-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*cos(p_2)*cos(p_3)) + ((-(-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*sin(p_2)*sin(p_3) + (-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*cos(p_2)*cos(p_3))*cos(p_4) - ((-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*sin(p_2)*cos(p_3) + (-sin(p_0)*sin(p_1) + cos(p_0)*cos(p_1))*sin(p_3)*cos(p_2))*sin(p_4))*(-m_0*cos(p_1)*cos(p_2 + p_3 + p_4) - m_1*cos(p_2 + p_3 + p_4) - m_2*cos(p_3 + p_4) - m_3*cos(p_4) + x_0*cos(p_0 + p_1)*cos(p_2 + p_3 + p_4) + x_1*sin(p_0 + p_1)*cos(p_2 + p_3 + p_4) - x_2*sin(p_2 + p_3 + p_4))],
[        m_0*sin(p_0) + m_1*(sin(p_0)*cos(p_1) + sin(p_1)*cos(p_0)) + m_2*(sin(p_0)*cos(p_1) + sin(p_1)*cos(p_0))*cos(p_2) + m_3*(-(sin(p_0)*cos(p_1) + sin(p_1)*cos(p_0))*sin(p_2)*sin(p_3) + (sin(p_0)*cos(p_1) + sin(p_1)*cos(p_0))*cos(p_2)*cos(p_3)) + ((-(sin(p_0

In [8]:
center = ((A+C)/2 + (B+D)/2)/2


In [9]:
# constraints of acute angles
c1 = (A - B).dot(A - M)  # to be positive
c2 = (B - A).dot(B - M)
c3 = (C - D).dot(C - M)
c4 = (D - C).dot(D - M)

# equality constraints. Keep contacts
eq_1 = sy.sqrt((M - N1).dot(M - N1)) - r[0] - r[1]
eq_2 = sy.sqrt((M - N2).dot(M - N2)) - r[0] - r[1]

# contact angle, N1_M_N2 > 90 degrees
c5 = - (M - N1).dot(M - N2)  # to be positive

## cost function
g = - c5 / sy.sqrt((M - N1).dot(M - N1))/sy.sqrt((M - N2).dot(M - N2))  # to be minimized, should be negative

# friction cone
mu = 0.5
cos_theta = np.cos(np.arctan(mu))
c6 = (M - N1).dot(N2 - N1) / sy.sqrt((M - N1).dot(M - N1))/ sy.sqrt((N2 - N1).dot(N2 - N1)) - cos_theta
c7 = (M - N2).dot(N1 - N2) / sy.sqrt((M - N2).dot(M - N2))/sy.sqrt((N2 - N1).dot(N2 - N1)) - cos_theta








In [10]:
T_all[0]

Matrix([
[cos(q_0), -sin(q_0), 0, l_0*cos(q_0)],
[sin(q_0),  cos(q_0), 0, l_0*sin(q_0)],
[       0,         0, 1,            0],
[       0,         0, 0,            1]])

In [11]:
dof

5

In [12]:
## self-collision avoidance. Todo
c_collision = []
for i in range(dof):
    A1 = T_all[i][:3, 3]
    A2 = T_all_j[i][:3, 3]
    c_i = sy.sqrt((A1 - A2).dot(A1 - A2))  - 2 * r[0]
    c_collision.append(c_i)


In [13]:
## self-collision avoidance. Todo
# get the perpendicular line for AB and CD, it should be longer that 2 * r
n34 = sy.symbols('alpha_:2')
N3 = A + (B-A) * n34[0]
N4 = C + (D-C) * n34[1]
Eq_n3 =  (B-A).dot(N3-N4)
Eq_n4 =  (C-D).dot(N3-N4)
# sy.solve([Eq_n3, Eq_n4], n34, dict=False)



In [14]:
sy.symbols('alpha_:2')

(alpha_0, alpha_1)

In [15]:
n34

(alpha_0, alpha_1)

In [16]:
from hand_sym import subs_value

In [17]:
(1,) + q[1:]

(1, q_1, q_2, q_3, q_4)

In [18]:
l_ = [0.05]*5     # link length
m_ = [0.05]*5     # link length
r_ = [0.01, 0.02]  # radius
q0 = 0


p0 = np.pi/4   # angle for finger placement


c = [c1, c2, c3, c4, c6, c7] + c_collision
qpx = q[1:] + p[1:] + x
subs_dict = subs_value([q, p,x,l,m,r], [(q0,) + q[1:], (p0,) + p[1:], x, l_, m_, r_])


c_ = [sy.lambdify([qpx], tmp.subs(subs_dict)) for tmp in c]
eqs = [eq_1, eq_2]
g_ = sy.lambdify([qpx], g.subs(subs_dict))
eqs_ = [sy.lambdify([qpx], tmp.subs(subs_dict)) for tmp in eqs]


# dis_sum = sy.sqrt(eq_1 **2 + eq_2 **2)
# dis_sum = eq_1 **2 + eq_2 **2

dis_sum = (eq_1 **2 + eq_2 **2)*1000 - c5
dis_sum_ = sy.lambdify([qpx], dis_sum.subs(subs_dict))

dis_sum_jac = sy.Matrix([dis_sum.subs(subs_dict)]).jacobian(qpx)
dis_sum_jac_ = sy.lambdify([qpx], dis_sum_jac)



center_ = sy.lambdify([qpx[:8]], center.subs(subs_dict))




In [19]:
dis_sum

1000*(-r_0 - r_1 + sqrt((l_2*sin(q_2) + x_2 - (-sin(q_2)*cos(q_3) - sin(q_3)*cos(q_2))*(-l_0*cos(q_1)*cos(q_2 + q_3) - l_1*cos(q_2 + q_3) - l_2*cos(q_3) + x_0*cos(q_0 + q_1)*cos(q_2 + q_3) + x_1*sin(q_0 + q_1)*cos(q_2 + q_3) - x_2*sin(q_2 + q_3)))**2 + (-l_0*sin(q_0) - l_1*(sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0)) - l_2*(sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0))*cos(q_2) + x_1 - (-(sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0))*sin(q_2)*sin(q_3) + (sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0))*cos(q_2)*cos(q_3))*(-l_0*cos(q_1)*cos(q_2 + q_3) - l_1*cos(q_2 + q_3) - l_2*cos(q_3) + x_0*cos(q_0 + q_1)*cos(q_2 + q_3) + x_1*sin(q_0 + q_1)*cos(q_2 + q_3) - x_2*sin(q_2 + q_3)))**2 + (-l_0*cos(q_0) - l_1*(-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1)) - l_2*(-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1))*cos(q_2) + x_0 - (-(-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1))*sin(q_2)*sin(q_3) + (-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1))*cos(q_2)*cos(q_3))*(-l_0*cos(q_1)*cos(q_2 + q_3) - l_1*cos(q_2 + q_3) - l_2*cos(q_3) + x_0*cos(q_0 + q_1)*

In [20]:
c_[4]([0.1]*4 + [0.1]*4 + [0.05]*3)  # test the value, using 24 µs

0.03474583996653735

In [21]:
g_([0.1]*4 + [0.1]*4 + [0.05]*3) 

0.689104702231812

In [22]:
eqs_[0]([0.1]*4 + [0.1]*4 + [0.05]*3)


0.030868546439511264

In [23]:
# optimization
cons = ()
for k in c_:
    cons += ({'type': 'ineq', 'fun': k},)
# for k in eqs_:
#     cons += ({'type': 'eq', 'fun': k},)

# bounds
joint_bnds = ((-np.pi/2, np.pi/2),(-np.pi/2, np.pi/2),(-np.pi/2, np.pi/2),(-np.pi/2, np.pi/2))
xyz_bounds = ((0, l_[0]*5), (0, l_[0]*5), (-l_[0]*5 , l_[0]*5))
bnds = joint_bnds + joint_bnds + xyz_bounds


alpha = 10

# cost function
def cos_fun(x):
    # return g_(x)
    return dis_sum_(x) * alpha

# Jacobian of cost function
def jac(x):
    return dis_sum_jac_(x) * alpha


In [24]:
# visualize the grasping synthesis in MuJoCo
import gait_test.hand_generator_from_genes as hg
import mujoco
from mujoco import viewer
from controller_full import Robot

fingers = [1,1] + [0] *6
dofs = [4] * 8
link_lengths = [l_[0], m_[0]] + [1] * 6

hand = hg.crawling_hand(fingers, dofs, link_lengths, objects=True, d_angle=p0)  # build the robot xml file
xml_data = hand.return_xml()
model = mujoco.MjModel.from_xml_string(xml_data)
data = mujoco.MjData(model)

view = viewer.launch_passive(model, data)
robot = Robot(model, data, view, fingers, dofs, auto_sync=True, obj_names=['sphere_1'])



In [25]:
from scipy.optimize import minimize

x0 = [0.1]*4 + [0.1]*4 + [0.05,0.05,0]
x0 = [0]*8 + [0.225, 0.1, 0.01]
q_init = np.ones(8) * 0.01 
x0 = center_(q_init).flatten()
x0 = np.concatenate([q_init, x0])
res = minimize(dis_sum_, x0, jac=dis_sum_jac_, method='SLSQP',  bounds=bnds, constraints=cons, options={'maxiter':1000, 'disp':1})  #  0.45 s with a good initial guess


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.0008915246684785613
            Iterations: 27
            Function evaluations: 45
            Gradient evaluations: 27


In [26]:
print(eqs_[0](res.x))
print(eqs_[1](res.x))

1.3016743010886567e-05
3.16070649583923e-05


In [27]:
robot.d.qpos[2] = 0.3
position = res.x[-3:] + robot.x[:3]
robot.modify_first_finger(res.x[:8])

obj_name = 'sphere_1'
robot.modify_obj_pose(obj_name, position)


In [28]:
eq_1_ = sy.lambdify([qpx], eq_1.subs(subs_dict))
eq_1_(res.x)

1.3016743010886567e-05

In [29]:
position

array([0.18280354, 0.07557834, 0.30174097])

In [30]:
# import xml.etree.ElementTree as ET
# 
# root = ET.fromstring(xml_data)
# with open("bookstore.xml", "w") as file:
#     file.write(ET.tostring(root, encoding='unicode'))

In [31]:
from grasping_synthesis import grasping

g1 = grasping()
l_ = [0.05]*5     # link length
m_ = [0.05]*5     # link length
r_ = [0.01, 0.02]  # radius
q0 = 0


p0 = np.pi/4   # angle for finger placement
g1.generate_grasp(l_,m_,r_,q0,p0,)


     fun: -0.0008944498310629659
     jac: array([-2.23339114e-03, -2.37000745e-04, -1.48088520e-04, -5.99641452e-05,
       -3.82024930e-03,  5.03944058e-05,  2.78561837e-05,  5.26776409e-06,
       -1.53007375e-02,  3.04195802e-02, -1.43646304e-03])
 message: 'Optimization terminated successfully'
    nfev: 41
     nit: 24
    njev: 24
  status: 0
 success: True
       x: array([ 0.39688331,  0.22125101,  0.02427272, -0.38741435, -0.26673268,
        0.08353305,  0.06912879,  0.0015972 ,  0.20386   ,  0.09703196,
       -0.01783123])